In [55]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
# display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set(style = "whitegrid")        

encoding="latin1"

Why?

files often fail UTF-8

Latin1 safely loads international text

In [57]:
# Load the dataset
data = pd.read_csv(r"C:\Data science work\Projects\SupplyChain Analytics Project\Data\supplychaindata.csv", encoding='latin1') 

In [58]:
data.shape

(180519, 53)

In [59]:
data.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De

In [60]:
#We remove junk and focus only on supply chain KPIs.

col =[
    "Order Id",
    "order date (DateOrders)",
    "Shipping Mode",
    "Days for shipping (real)",
    "Days for shipment (scheduled)",
    "Delivery Status",
    "Late_delivery_risk",
    "Sales per customer",
    "Benefit per order",
    "Order City",
    "Order Country",
    "Category Name",
    "Product Name" 
]
data_sc = data[col].copy() # Creating a new dataframe for supply chain analysis
data_sc.head()


,Order Id,order date (DateOrders),Shipping Mode,Days for shipping (real),Days for shipment (scheduled),Delivery Status,Late_delivery_risk,Sales per customer,Benefit per order,Order City,Order Country,Category Name,Product Name
0,77202,1/31/2018 22:56,Standard Class,3,4,Advance shipping,0,314.64,91.25,Bekasi,Indonesia,Sporting Goods,Smart watch
1,75939,1/13/2018 12:27,Standard Class,5,4,Late delivery,1,311.36,-249.09,Bikaner,India,Sporting Goods,Smart watch
2,75938,1/13/2018 12:06,Standard Class,4,4,Shipping on time,0,309.72,-247.78,Bikaner,India,Sporting Goods,Smart watch
3,75937,1/13/2018 11:45,Standard Class,3,4,Advance shipping,0,304.81,22.86,Townsville,Australia,Sporting Goods,Smart watch
4,75936,1/13/2018 11:24,Standard Class,2,4,Advance shipping,0,298.25,134.21,Townsville,Australia,Sporting Goods,Smart watch


In [61]:
data_sc.dtypes

Order Id                           int64
order date (DateOrders)           object
Shipping Mode                     object
Days for shipping (real)           int64
Days for shipment (scheduled)      int64
Delivery Status                   object
Late_delivery_risk                 int64
Sales per customer               float64
Benefit per order                float64
Order City                        object
Order Country                     object
Category Name                     object
Product Name                      object
dtype: object

Why?

Enables monthly demand analysis

Needed for trends and forecasting

data always has messy dates

In [62]:
# fix date column ↦ Convert to datetime format

data_sc["order date (DateOrders)"] = pd.to_datetime(data_sc["order date (DateOrders)"], errors = "coerce") 

In [63]:
data_sc.head()

,Order Id,order date (DateOrders),Shipping Mode,Days for shipping (real),Days for shipment (scheduled),Delivery Status,Late_delivery_risk,Sales per customer,Benefit per order,Order City,Order Country,Category Name,Product Name
0,77202,2018-01-31 22:56:00,Standard Class,3,4,Advance shipping,0,314.64,91.25,Bekasi,Indonesia,Sporting Goods,Smart watch
1,75939,2018-01-13 12:27:00,Standard Class,5,4,Late delivery,1,311.36,-249.09,Bikaner,India,Sporting Goods,Smart watch
2,75938,2018-01-13 12:06:00,Standard Class,4,4,Shipping on time,0,309.72,-247.78,Bikaner,India,Sporting Goods,Smart watch
3,75937,2018-01-13 11:45:00,Standard Class,3,4,Advance shipping,0,304.81,22.86,Townsville,Australia,Sporting Goods,Smart watch
4,75936,2018-01-13 11:24:00,Standard Class,2,4,Advance shipping,0,298.25,134.21,Townsville,Australia,Sporting Goods,Smart watch


In [64]:
data_sc["order date (DateOrders)"].isnull().sum()

np.int64(0)

Why?

Missing shipping days = wrong KPIs

Missing profit = wrong cost analysis

In [65]:
data_sc.isnull().sum()

Order Id                         0
order date (DateOrders)          0
Shipping Mode                    0
Days for shipping (real)         0
Days for shipment (scheduled)    0
Delivery Status                  0
Late_delivery_risk               0
Sales per customer               0
Benefit per order                0
Order City                       0
Order Country                    0
Category Name                    0
Product Name                     0
dtype: int64

Delay_Days → delivery performance

Profit_Status → cost optimization

In [66]:
# Create Business Metrics

data_sc["Delay days"] = data_sc["Days for shipping (real)"] - data_sc["Days for shipment (scheduled)"] # Calculate delay in shipping

data_sc["profit_status"] =data_sc["Benefit per order"].apply(
    lambda x: "Profit" if x > 0 else "Loss"
)

In [67]:
# Create Time Columns

data_sc["order_year"] = data_sc["order date (DateOrders)"].dt.year # Extract year from date
data_sc["order_month"] = data_sc["order date (DateOrders)"].dt.month # Extract month from date
data_sc ["order_day"]= data_sc["order date (DateOrders)"].dt.day # Extract day from date    

In [68]:
# save cleaned data
data_sc.to_csv(r"C:\Data science work\Projects\SupplyChain Analytics Project\Data\supplychain_cleaned_data.csv", index = False)

In [69]:
data_sc.head()

,Order Id,order date (DateOrders),Shipping Mode,Days for shipping (real),Days for shipment (scheduled),Delivery Status,Late_delivery_risk,Sales per customer,Benefit per order,Order City,Order Country,Category Name,Product Name,Delay days,profit_status,order_year,order_month,order_day
0,77202,2018-01-31 22:56:00,Standard Class,3,4,Advance shipping,0,314.64,91.25,Bekasi,Indonesia,Sporting Goods,Smart watch,-1,Profit,2018,1,31
1,75939,2018-01-13 12:27:00,Standard Class,5,4,Late delivery,1,311.36,-249.09,Bikaner,India,Sporting Goods,Smart watch,1,Loss,2018,1,13
2,75938,2018-01-13 12:06:00,Standard Class,4,4,Shipping on time,0,309.72,-247.78,Bikaner,India,Sporting Goods,Smart watch,0,Loss,2018,1,13
3,75937,2018-01-13 11:45:00,Standard Class,3,4,Advance shipping,0,304.81,22.86,Townsville,Australia,Sporting Goods,Smart watch,-1,Profit,2018,1,13
4,75936,2018-01-13 11:24:00,Standard Class,2,4,Advance shipping,0,298.25,134.21,Townsville,Australia,Sporting Goods,Smart watch,-2,Profit,2018,1,13
